In [199]:
# 라이브러리 설정
# 1. 라이브러리 설정
import numpy as np
import random
import pandas as pd
import statistics
import matplotlib.pyplot as plt
# %matplotlib inline

from sklearn.metrics.pairwise import cosine_similarity
import operator

from tensorflow.keras.layers import Input, Embedding, Dot, Reshape, Dense, Concatenate
from tensorflow.keras.models import Model
import os 
from tensorflow.keras.callbacks import ModelCheckpoint
# from umap import UMAP


In [200]:
# 2. 데이터 불러오기 & 확인 & 시각화
datapath = '/content/drive/MyDrive'
anime = pd.read_csv(datapath + '/anime/anime.csv')
rating = pd.read_csv(datapath + '/anime/rating.csv')


# 데이터 시각화
# 콘텐츠를 평가한 유저 숫자
rating_per_user = rating.groupby('user_id')['rating'].count()
statistics.mean(rating_per_user.tolist()) # 1인당 106.28765558049378

# 콘텐츠별 평가된 평점 등급
ratings_per_anime = rating.groupby('anime_id')['rating'].count()
statistics.mean(ratings_per_anime.tolist()) # 1콘텐츠당 697.3번 리뷰

697.6550892857143

In [201]:
# 3. 데이터 전처리
# rating 평점 '-1' -> '5'으로 변경 = normal
# -1 : 보았으나 평가하지 않은 데이터
# rating = rating.replace({'rating':-1},5)/


# rating(평가된 콘텐츠 전처리)
# 너무 적게 평가된 콘텐츠 제거(50개 이하 평가 데이터 제거)
ratings_per_anime_df = pd.DataFrame(ratings_per_anime)

filtered_ratings_per_anime_df = ratings_per_anime_df[ratings_per_anime_df.rating >= 50]
popular_anime = filtered_ratings_per_anime_df.index.tolist()

# anime(콘텐츠를 평가한 유저 데이터 처리)
# 너무 적게 평가한 유저 제거 -> 30개 이하 평가한 유저 제거
rating_per_user_df = pd.DataFrame(rating_per_user)
filtered_rating_per_user_df = rating_per_user_df[rating_per_user_df.rating >= 30]
rating_users = filtered_rating_per_user_df.index.tolist()

filtered_rating = rating[rating.anime_id.isin(popular_anime)]
filtered_rating = rating[rating.user_id.isin(rating_users)]

# data merge on=anime_id
filtered_rating = pd.merge(filtered_rating, anime, on='anime_id')
filtered_rating = filtered_rating.iloc[:,[0,1,2,3]]
filtered_rating['anime_id'] = filtered_rating['name']
filtered_rating = filtered_rating.drop('name', axis=1)
filtered_rating = filtered_rating.sample(frac=1)
# filtered_rating['rating_x'] = filtered_rating['rating_x'] / 10

print(filtered_rating[:50])

         user_id                                           anime_id  rating_x
4814878     1522                                          Beelzebub         8
6940866    38479                                  Nisekoimonogatari         7
3609692    37635                                           Nisekoi:         9
2437939    51418                Baka to Test to Shoukanjuu: Matsuri         7
4373197    67226                                       Kanon (2006)         9
6286698    19274                     Kiniro no Corda: Secondo Passo         9
6691383    37840                           Final Fantasy: Unlimited        -1
1413923    12415                                           Noragami         8
650301     62670                      Dakara Boku wa, H ga Dekinai.         8
4493590    35025                      Suzumiya Haruhi no Shoushitsu        -1
41353      18708                                         Elfen Lied         8
4025301      151                                       Black Lag

In [202]:
# 4. 임베딩 모델 만들기
# n = 500000
# df_train = filtered_rating[:-n]
# df_test = filtered_rating[-n:]

# 코드 분석 + 변환 필요

# user/movie_id (1, user_id) (1, movie_id) ~ enumerate mapping
user_id_mapping = {id:i for i, id in enumerate(filtered_rating['user_id'].unique())}
id_user_mapping = {i: id for id, i in user_id_mapping.items()}
anime_id_mapping = {id:i for i, id in enumerate(filtered_rating['anime_id'].unique())}
id_anime_mapping = {i: id for id, i in anime_id_mapping.items()}



# Create correctly mapped train- & testset
train_user_data = filtered_rating['user_id'].map(user_id_mapping)
train_movie_data = filtered_rating['anime_id'].map(anime_id_mapping)

test_user_data = filtered_rating['user_id'].map(user_id_mapping)
test_movie_data = filtered_rating['anime_id'].map(anime_id_mapping)

# mapping data load : user/movie_id_mapping length
users = len(user_id_mapping)
movies = len(anime_id_mapping)

# embedding size setup
user_embedding_size = 20
movie_embedding_size = 20


##### Create model
# Set input layers
user_id_input = Input(shape=[1], name='user_input')
movie_id_input = Input(shape=[1], name='anime_input')

# Create embedding layers for users and movies
user_embedding = Embedding(output_dim=user_embedding_size, 
                           input_dim=users,
                           input_length=1, 
                           name='user_embedding')(user_id_input)
movie_embedding = Embedding(output_dim=movie_embedding_size, 
                            input_dim=movies,
                            input_length=1, 
                            name='anime_embedding')(movie_id_input)

# Reshape the embedding layers
user_vector = Reshape([user_embedding_size])(user_embedding)
movie_vector = Reshape([movie_embedding_size])(movie_embedding)

# Concatenate the reshaped embedding layers
concat = Concatenate()([user_vector, movie_vector])

# Combine with dense layers
dense = Dense(256)(concat)
out = Dense(1)(dense)

# Setup model
# input : user_id_input, movie_id_input
model = Model(inputs=[user_id_input, movie_id_input], outputs=out)
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 anime_input (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 user_embedding (Embedding)     (None, 1, 20)        967320      ['user_input[0][0]']             
                                                                                                  
 anime_embedding (Embedding)    (None, 1, 20)        223920      ['anime_input[0][0]']            
                                                                                            

In [203]:
# 저장 위치 설정

# MODEL_DIR = '/home/juno/workspace/user_collaborative_filtering/model_save'
# if not os.path.exists(MODEL_DIR):
#     os.mkdir(MODEL_DIR)

# modelpath = '/home/juno/workspace/user_collaborative_filtering/model_save/save_best_only/anime_embedding_model{epoch:04d}-{val_loss:.4f}.h5'
# checkpointer = ModelCheckpoint(moniter='val_loss', vallbose=1, save_best_only=True)


# 모델 훈련
# validation_split : train 80% val 20%
history = model.fit([train_user_data, train_movie_data],
                    filtered_rating['rating_x'],
                    batch_size=8192, 
                    epochs=10,
                    validation_split=0.2,
                    shuffle=True
                    )

# model save, history save
# model.save('/home/juno/workspace/user_collaborative_filtering/model_save/best_model_history/anime_embedding_model1.h5')
# np.save('/home/juno/workspace/user_collaborative_filtering/model_save/best_model_history/anime_embedding_model1.npy', history.history)

Epoch 1/10
734/734 [==============================] - 23s 31ms/step - loss: 7.2912 - accuracy: 0.0021 - val_loss: 4.8484 - val_accuracy: 0.0020
Epoch 2/10
734/734 [==============================] - 22s 30ms/step - loss: 4.8063 - accuracy: 0.0021 - val_loss: 4.8171 - val_accuracy: 0.0020
Epoch 3/10
734/734 [==============================] - 22s 30ms/step - loss: 4.7751 - accuracy: 0.0021 - val_loss: 4.7933 - val_accuracy: 0.0020
Epoch 4/10
734/734 [==============================] - 22s 30ms/step - loss: 4.7566 - accuracy: 0.0021 - val_loss: 4.7836 - val_accuracy: 0.0020
Epoch 5/10
734/734 [==============================] - 22s 30ms/step - loss: 4.7444 - accuracy: 0.0021 - val_loss: 4.7757 - val_accuracy: 0.0020
Epoch 6/10
734/734 [==============================] - 22s 30ms/step - loss: 4.7366 - accuracy: 0.0021 - val_loss: 4.7705 - val_accuracy: 0.0020
Epoch 7/10
734/734 [==============================] - 22s 30ms/step - loss: 4.7299 - accuracy: 0.0021 - val_loss: 4.7646 - val_accuracy:

In [204]:
## user/anime extract embedding & analyze
import numpy as np
user_layer = model.get_layer('user_embedding')
# code 분석
user_weights = user_layer.get_weights()[0]
user_weights.shape

# print(user_weights)
user_weights = user_weights / np.linalg.norm(user_weights, axis = 1).reshape((-1, 1))
print(user_weights[0][:50])
print(np.sum(np.square(user_weights[0])))





[ 0.15963939  0.26928884 -0.06259102  0.3365219  -0.12593585  0.11188467
 -0.10724259 -0.30697787 -0.29854727  0.08142328 -0.1425078  -0.21400721
  0.20844598  0.29659885  0.2753743   0.18996558  0.24770696  0.29964748
 -0.30329832 -0.04874682]
1.0


In [205]:
anime_layer = model.get_layer('anime_embedding')
# code 분석
anime_weights = anime_layer.get_weights()[0]
anime_weights.shape

# print(user_weights)
anime_weights = anime_weights / np.linalg.norm(anime_weights, axis = 1).reshape((-1, 1))
print(anime_weights[0][:50])
print(np.sum(np.square(anime_weights[0])))


[ 0.16214794  0.28866085 -0.14391086  0.20612445  0.1633582   0.21133168
 -0.2634991   0.29573572 -0.28597024 -0.24566701  0.21102837  0.14316328
  0.22996569  0.12340849  0.2515198   0.19855796  0.28194672  0.19357598
 -0.30642647  0.10061302]
1.0000001


In [206]:
# user_id mapping / id_user_mapping

user_id_mapping = {id:i for i, id in enumerate(filtered_rating['user_id'].unique())}
id_user_mapping = {i: id for id, i in user_id_mapping.items()}

In [207]:
anime_id_mapping = {id:i for i, id in enumerate(filtered_rating['anime_id'].unique())}
id_anime_mapping = {i: id for id, i in anime_id_mapping.items()}

In [269]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 15

def find_similar(name, weights, index_name, n = 10, least = False, return_dist = False):
    
    # Select index and reverse index
    if index_name == 'user_weight':
        index = user_id_mapping
        rindex = id_user_mapping
    elif index_name == 'anime_id':
        index = anime_id_mapping
        rindex = id_anime_mapping
        
    # elif index_name == 'anime':
    #     index = anime_id_mapping
    #     rindex = id_anime_mapping

    
    # Check to make sure `name` is in index
    try:
        # Calculate dot product between book and all others
        dists = np.dot(weights, weights[index[name]])
    except KeyError:
        print(f'{name} Not Found.')
        return
    
    # Sort distance indexes from smallest to largest
    sorted_dists = np.argsort(dists)

     # If specified, find the least similar
    if least:
        # Take the first n from sorted distances
        closest = sorted_dists[:n]
         
        print(f'{index_name.capitalize()} furthest from {name}.\n')
        
    # Otherwise find the most similar
    else:
        # Take the last n sorted distances
        closest = sorted_dists[-n:]
        
        # Need distances later on
        if return_dist:
            return dists, closest
        
        
        print(f'{index_name.capitalize()} closest to {name}.\n')
        
    # Need distances later on
    if return_dist:
        return dists, closest
    
    
    # Print formatting
    max_width = max([len(rindex[c]) for c in closest])
    
    # Print the most similar and distances
    for c in reversed(closest):
        print(f'{index_name.capitalize()}: {rindex[c]:{max_width + 2}} Similarity: {dists[c]:.{2}}')

In [270]:
find_similar('Digimon Adventure 02 Movies', anime_weights, 'anime_id')

Anime_id closest to Digimon Adventure 02 Movies.

Anime_id: Digimon Adventure 02 Movies        Similarity: 1.0
Anime_id: Mameshiba Bangai-hen               Similarity: 0.81
Anime_id: Nobara no Julie                    Similarity: 0.71
Anime_id: Nezumi-kun no Chokki               Similarity: 0.68
Anime_id: Shisei Sasshin                     Similarity: 0.68
Anime_id: Absolute Duo                       Similarity: 0.67
Anime_id: Ninja Hattori-kun (2012) Special   Similarity: 0.66
Anime_id: Recorder to Randoseru Do♪          Similarity: 0.66
Anime_id: Boyfriend                          Similarity: 0.65
Anime_id: Ring ni Kakero 1                   Similarity: 0.65


In [278]:
find_similar('63878', user_weights, 'user_weight')

63878 Not Found.


In [279]:
!pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [280]:
from umap import UMAP
import matplotlib.pyplot as plt

In [281]:
def reduce_dim(weights, components = 2):
    if weights == 'user_weights':
      return UMAP(n_components=components, metric = 'cosine', init = 'random', n_neighbors = 15).fit_transform(weights)

    elif weights == 'anime_weights':
      return UMAP(n_components=components, metric = 'cosine', init = 'random', n_neighbors = 15).fit_transform(weights)

In [283]:
user_dim = reduce_dim(anime_weights, components = 2)

plt.figure(figsize = (10, 8))
plt.plot(user_dim[:, 0], user_dim[:, 1], 'g.');
plt.xlabel('UMAP 1'); plt.ylabel('UMAP 2'); plt.title('anime Embeddings Visualized with UMAP');

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """


TypeError: ignored

<Figure size 720x576 with 0 Axes>

In [284]:
anime_dim = reduce_dim(anime_weights, components = 2)

plt.figure(figsize = (10, 8))
plt.plot(user_dim[:, 0], user_dim[:, 1], 'g.');
plt.xlabel('UMAP 1'); plt.ylabel('UMAP 2'); plt.title('anime Embeddings Visualized with UMAP');

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """


TypeError: ignored

<Figure size 720x576 with 0 Axes>

In [285]:
def plot_closest(item, weights, index_name, n, plot_data):

    # Find the closest items
    dist, closest = find_similar(item, weights, index_name, n, return_dist=True)
    
    # Choose mapping for look up
    if index_name == 'user_weights':
        index = user_id_mapping
        rindex = id_user_mapping
    elif index_name == 'anime_weights':
        index = anime_id_mapping
        rindex = id_anime_mapping
    
    plt.figure(figsize = (10, 9))
    plt.rcParams['font.size'] = 14
    
    # Limit distances
    dist = dist[closest]
    
    # Plot all of the data
    plt.scatter(plot_data[:, 0], plot_data[:, 1], alpha = 0.1, color = 'goldenrod')
    
    # Plot the item
    plt.scatter(plot_data[closest[-1], 0], plot_data[closest[-1], 1], s = 600, edgecolor = 'k', color = 'forestgreen')
    
    # Plot the closest items
    p = plt.scatter(plot_data[closest[:-1], 0], plot_data[closest[:-1], 1], 
                c = dist[:-1], cmap = plt.cm.RdBu_r, s = 200, alpha = 1, marker = '*')
    
    # Colorbar management
    cbar = plt.colorbar()
    cbar.set_ticks([])
    
    tick_labels = []

    # Tick labeling for colorbar
    for idx, distance in zip(closest[:-1], dist[:-1]):
        name_str = ''
        for word in rindex[idx].split():
            # Title uses latex for italize
            name_str += ' $\it{' + word + '}$'
        name_str += ': ' + str(round(distance, 2))
        tick_labels.append(name_str)
    
    for j, lab in enumerate(tick_labels):
        cbar.ax.text(1, (2 * j + 1) / ((n - 1) * 2), lab, ha='left', va='center', size = 12)
    cbar.ax.set_title(f'{index_name.capitalize()} and Cosine Distance', loc = 'left', size = 14)
    
    # Formatting for italicized title
    name_str = f'{index_name.capitalize()}s Most Similar to'
    for word in item.split():
        # Title uses latex for italize
        name_str += ' $\it{' + word + '}$'
    
    # Labeling
    plt.xlabel('UMAP 1'); plt.ylabel('UMAP 2'); 
    plt.title(name_str);

In [286]:
plot_closest('Btooom!', anime_weights, 'anime_weights', 20, anime_dim)

UnboundLocalError: ignored

In [130]:
plot_closest('The Last: Naruto the Movie', anime_weights, 'anime_weights', 20, anime_dim)

In [ ]:
user_id_mapping = {id:i for i, id in enumerate(filtered_rating['user_id'].unique())}
id_user_mapping = {i:id for id, i in user_id_mapping.items()}

In [ ]:
def find_similar(name, weights, index_name = 'user_id', n = 10, least = False, return_dist = False, plot = False):
    
    index_name == 'user'
    index = user_id_mapping
    rindex = id_user_mapping 

    
    # Check to make sure `name` is in index
    dists = np.dot(weights, weights[index[name]])
    
    # Sort distance indexes from smallest to largest
    sorted_dists = np.argsort(dists)
    
    # Plot results if specified
    if plot:
        
        # Find furthest and closest items
        furthest = sorted_dists[:(n // 2)]
        closest = sorted_dists[-n-1: len(dists) - 1]
        items = [rindex[c] for c in furthest]
        items.extend(rindex[c] for c in closest)
        
        # Find furthest and closets distances
        distances = [dists[c] for c in furthest]
        distances.extend(dists[c] for c in closest)
        
        colors = ['r' for _ in range(n //2)]
        colors.extend('g' for _ in range(n))
        
        data = pd.DataFrame({'distance': distances}, index = items)
        
        # Horizontal bar chart
        data['distance'].plot.barh(color = colors, figsize = (10, 8),
                                   edgecolor = 'k', linewidth = 2)
        plt.xlabel('Cosine Similarity');
        plt.axvline(x = 0, color = 'k');
        
        # Formatting for italicized title
        name_str = f'{index_name.capitalize()}s Most and Least Similar to'
        for word in name.split():
            # Title uses latex for italize
            name_str += ' $\it{' + word + '}$'
        plt.title(name_str, x = 0.2, size = 28, y = 1.05)
        
        return None
    
    # If specified, find the least similar
    if least:
        # Take the first n from sorted distances
        closest = sorted_dists[:n]
         
        print(f'{index_name.capitalize()}s furthest from {name}.\n')
        
    # Otherwise find the most similar
    else:
        # Take the last n sorted distances
        closest = sorted_dists[-n:]
        
        # Need distances later on
        if return_dist:
            return dists, closest
        
        
        print(f'{index_name.capitalize()}s closest to {name}.\n')
        
    # Need distances later on
    if return_dist:
        return dists, closest
    
    
    # Print formatting
    max_width = max([len(rindex[c]) for c in closest])
    
    # Print the most similar and distances
    for c in reversed(closest):
        print(f'{index_name.capitalize()}: {rindex[c]:{max_width + 2}} Similarity: {dists[c]:.{2}}')